# Capstone Project: selamat pagi anak-anakku

- Amelia Yunisa
- Mochamad Bayu Safutra
- Rio Ferdinand
- Rizka Salisa Puteri

## Import Library

In [1]:
import os, glob, random, shutil
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import tensorflow as tf
# import tensorflowjs as tfjs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback 

## Load Dataset Resep

In [2]:
df = pd.read_csv('dataset_resep.csv')

print(df.head())

print(df.info())

# print(df.describe())

print(df.isnull().sum())
print(df.duplicated().sum())

              Nama                                          Deskripsi  \
0      Ayam Goreng  Ayam Goreng adalah hidangan tradisional Indone...   
1        Ayama Pop  Ayam Pop adalah hidangan tradisional Indonesia...   
2            Bakso  Bakso adalah hidangan tradisional Indonesia ya...   
3  Dendeng Batokok  Dendeng merupakan daging kering yang diiris ti...   
4             Gado  Gado-gado adalah hidangan tradisional Indonesi...   

                                               Resep  
0  Bahan-bahan:\n1 kg Ayam, cuci bersih dan poton...  
1  Bahan-bahan :\n1 ekor ayam kampung (bisa pake ...  
2  Bahan-bahan\n250 gr Fillet ayam bagian paha (s...  
3  Bahan-bahan:\n1 kg daging sapi\n1 ons Cabai me...  
4  Bahan-bahan:\n1 buah timun (iris tipis)\n2 tan...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nama       32 non-null     object
 1   Deskrip

## Splitting Data

In [3]:
import os, glob, random, shutil

src_root  = 'food'
dst_root  = 'food_split'
splits    = {'train': .7, 'validation': .15, 'test': .15}

# 1) Ambil daftar kelas dari src_root/train
classes = os.listdir(os.path.join(src_root, 'train'))

# 2) Buat direktori tujuan per split & per kelas
for split in splits:
    for cls in classes:
        os.makedirs(os.path.join(dst_root, split, cls), exist_ok=True)

# 3) Gabungkan file per kelas dari train/validation/test
for cls in classes:
    all_files = []
    for sub in ['train','validation','test']:
        pattern = os.path.join(src_root, sub, cls, '*')
        all_files += glob.glob(pattern)

    random.seed(42)
    random.shuffle(all_files)
    n = len(all_files)
    n_train = int(splits['train'] * n)
    n_val   = int(splits['validation']   * n)

    train_files = all_files[:n_train]
    val_files   = all_files[n_train:n_train+n_val]
    test_files  = all_files[n_train+n_val:]

    # 4) Copy ke folder baru
    for f in train_files:
        shutil.copy(f, os.path.join(dst_root, 'train', cls))
    for f in val_files:
        shutil.copy(f, os.path.join(dst_root, 'validation', cls))
    for f in test_files:
        shutil.copy(f, os.path.join(dst_root, 'test', cls))

    print(f"{cls}: train={len(train_files)}, validation={len(val_files)}, test={len(test_files)}")


ayam_goreng: train=74, validation=16, test=17
ayam_pop: train=79, validation=16, test=18
bakso: train=1507, validation=323, test=324
dendeng_batokok: train=76, validation=16, test=17
gado: train=1501, validation=321, test=323
grontol: train=210, validation=45, test=45
gudeg: train=328, validation=70, test=71
gulai_ikan: train=77, validation=16, test=18
gulai_tambusu: train=72, validation=15, test=16
gulai_tunjang: train=83, validation=17, test=19
kue_ape: train=101, validation=21, test=23
kue_bika_ambon: train=142, validation=30, test=32
kue_cenil: train=142, validation=30, test=31
kue_dadar_gulung: train=162, validation=34, test=36
kue_gethuk_lindri: train=154, validation=33, test=33
kue_kastengel: train=154, validation=33, test=33
kue_klepon: train=168, validation=36, test=36
kue_lapis: train=168, validation=36, test=37
kue_lemper: train=125, validation=27, test=28
kue_lumpur: train=173, validation=37, test=38
kue_nagasari: train=161, validation=34, test=35
kue_pastel: train=161, val

## Load Dataset

In [4]:
# 1. Atur path dan parameter global
base_dir   = os.getcwd()
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

## Augmentasi Data

In [9]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

#    b) Hanya rescale untuk validation dan test
val_test_gen = ImageDataGenerator(rescale=1./255)

# 3. Buat generator
train_generator = train_gen.flow_from_directory(
    os.path.join(dst_root, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', 
    shuffle=True
)

validation_generator = val_test_gen.flow_from_directory(
    os.path.join(dst_root, 'validation'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_gen.flow_from_directory(
    os.path.join(dst_root, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 8695 images belonging to 32 classes.
Found 1880 images belonging to 32 classes.
Found 1912 images belonging to 32 classes.


## Modeling

In [11]:
# 1. Load Pretrained MobileNetV2
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

# 2. Bangun model transfer learning kamu
model = Sequential([
    base_model,

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')  # jumlah kelas dari generator
])

# 3. Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4. Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(patience=3, factor=0.2, monitor='val_loss')

# 5. Fit model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stop, reduce_lr]
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 32)       │       368,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 3, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,657,600 (10.14 MB)

 Trainable params: 399,616 (1.52 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 288s 1s/step - accuracy: 0.3942 - loss: 2.2587 - val_accuracy: 0.6096 - val_loss: 1.4158 - learning_rate: 0.0010
Epoch 2/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 285s 1s/step - accuracy: 0.6023 - loss: 1.4271 - val_accuracy: 0.7128 - val_loss: 1.0129 - learning_rate: 0.0010
Epoch 3/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 259s 952ms/step - accuracy: 0.6660 - loss: 1.1701 - val_accuracy: 0.7410 - val_loss: 0.8959 - learning_rate: 0.0010
Epoch 4/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 172s 632ms/step - accuracy: 0.6917 - loss: 1.0491 - val_accuracy: 0.7644 - val_loss: 0.8333 - learning_rate: 0.0010
Epoch 5/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 175s 644ms/step - accuracy: 0.7185 - loss: 0.9404 - val_accuracy: 0.7628 - val_loss: 0.8072 - learning_rate: 0.0010
Epoch 6/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 173s 636ms/step - accuracy: 0.7536 - loss: 0.8458 - val_accuracy: 0.7755 - val_loss: 0.8044 - learning_rate: 0.0010
Epoch 7/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 177s 652ms/step - accuracy: 0.

## Evaluation

## Inference